In [26]:
"""
При исполнении кода у меня почему-то возникли трудности с совместимостью версий Numpy и RDKit.
Поэтому пришлось добавить в код отдельный блок для решения этой проблемы.

Код был прогнан через DeepSeek для исправления неточностей, улучшения структуры,
решения проблемы с совместимостью библиотек.

Не знаю, насколько удобно, но вместо отдельных блоков, специально свела всё в сплошной код.
Ключевые этапы постаралась соблюсти, хотя для удобства объединяла функции.
Оставила по-минимуму комментариев к коду.

Добавила удаление дубликатов SMILES и статистику по операциям.

Не уверена, что с CheMBL спарсилось нужное количество записей.

"""
# Установка библиотек
!pip install numpy==1.23.5 pandas==1.5.3 scikit-learn==1.2.2
!pip install rdkit-pypi==2022.9.5 chembl-webresource-client==0.10.8
!pip install matplotlib==3.7.1 seaborn==0.12.2

# Импорт библиотек
import pandas as pd
import numpy as np
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Константы
TARGET_CHEMBL_ID = "CHEMBL206"
ACTIVITY_TYPE = "Ki"
UNIT = "nM"
FINGERPRINT_RADIUS = 2
FINGERPRINT_BITS = 1024
VARIANCE_THRESHOLD = 0.01
PCA_VARIANCE_THRESHOLD = 0.95

def print_header(text):
    """Печать заголовка этапа"""
    print("\n" + "="*50)
    print(text.upper())
    print("="*50)

# ───────────────────────── 1. Загрузка данных ─────────────────────────
def load_chembl_data():
    """Загрузка данных из ChEMBL API"""
    print_header("Загрузка данных из ChEMBL")

    activity = new_client.activity
    activities = activity.filter(
        target_chembl_id=TARGET_CHEMBL_ID,
        type=ACTIVITY_TYPE,
        standard_units=UNIT,
        relation="=",
        assay_type="B"
    ).only(
        'molecule_chembl_id',
        'canonical_smiles',
        'standard_value',
        'standard_units'
    )

    df = pd.DataFrame.from_dict(activities)
    print(f"Получено записей с сервера: {len(df)}")
    return df

# ───────────────────────── 2. Предобработка данных ─────────────────────────
def preprocess_data(df):
    """Предобработка данных и преобразование Ki в pKi"""
    print_header("Предобработка данных")

    if df.empty:
        print("Нет данных для обработки!")
        return df

    initial_count = len(df)

    # Очистка SMILES и удаление дубликатов
    df = df.copy()
    df['canonical_smiles'] = df['canonical_smiles'].str.strip().replace('', np.nan)
    df = df.dropna(subset=['canonical_smiles'])

    # Удаление дубликатов SMILES
    duplicates = df.duplicated(subset=['canonical_smiles'], keep='first').sum()
    df = df.drop_duplicates(subset=['canonical_smiles'], keep='first')

    # Обработка значений активности
    df['standard_value'] = pd.to_numeric(df['standard_value'], errors='coerce')
    invalid_values = df['standard_value'].isna().sum()
    df = df.dropna(subset=['standard_value'])

    # Фильтрация положительных значений
    non_positive = (df['standard_value'] <= 0).sum()
    df = df[df['standard_value'] > 0]

    # Преобразование в pKi
    df['pValue'] = -np.log10(df['standard_value'] * 1e-9)

    # Вывод статистики
    print(f"Исходное количество записей: {initial_count}")
    print(f"Удалено дубликатов SMILES: {duplicates}")
    print(f"Удалено записей с нечисловыми значениями: {invalid_values}")
    print(f"Удалено записей с неположительными значениями: {non_positive}")
    print(f"Осталось валидных записей: {len(df)}")
    print(f"Диапазон pKi: {df['pValue'].min():.2f} - {df['pValue'].max():.2f}")

    return df.reset_index(drop=True)

# ───────────────────────── 3. Генерация отпечатков Моргана ─────────────────────────
def generate_fingerprints(df):
    """Генерация 1024-битных отпечатков Моргана"""
    print_header("Генерация молекулярных отпечатков")

    if df.empty:
        print("Нет данных для обработки!")
        return pd.DataFrame(), np.array([]), None

    fps = []
    valid_idx = []
    invalid_smiles = 0

    for i, row in df.iterrows():
        mol = Chem.MolFromSmiles(row['canonical_smiles'])
        if mol is None:
            invalid_smiles += 1
            continue
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=FINGERPRINT_RADIUS, nBits=FINGERPRINT_BITS)
        fps.append(fp)
        valid_idx.append(i)

    # Конвертация в numpy array
    np_fps = np.zeros((len(fps), FINGERPRINT_BITS))
    for i, fp in enumerate(fps):
        DataStructs.ConvertToNumpyArray(fp, np_fps[i])

    # Фильтрация признаков
    selector = VarianceThreshold(threshold=VARIANCE_THRESHOLD)
    filtered = selector.fit_transform(np_fps)

    print(f"Успешно обработано молекул: {len(fps)}")
    print(f"Невалидных SMILES: {invalid_smiles}")
    print(f"Размерность до фильтрации: {FINGERPRINT_BITS} признаков")
    print(f"Размерность после фильтрации: {filtered.shape[1]} признаков")

    return df.iloc[valid_idx].reset_index(drop=True), filtered, selector

# ───────────────────────── 4. PCA и визуализация ─────────────────────────
def apply_pca(features):
    """Применение PCA с автоматическим выбором числа компонент"""
    print_header("Анализ главных компонент (PCA)")

    if features.size == 0:
        print("Нет данных для анализа!")
        return np.array([]), None, np.array([])

    # Стандартизация
    scaler = StandardScaler()
    scaled = scaler.fit_transform(features)

    # Определение числа компонент
    pca = PCA()
    pca.fit(scaled)
    cum_var = np.cumsum(pca.explained_variance_ratio_)
    n_comp = np.argmax(cum_var >= PCA_VARIANCE_THRESHOLD) + 1

    # Применение PCA
    pca = PCA(n_components=n_comp)
    pca_features = pca.fit_transform(scaled)

    print(f"Исходное количество признаков: {features.shape[1]}")
    print(f"Выбрано главных компонент: {n_comp}")
    print(f"Объясненная дисперсия: {cum_var[n_comp-1]:.1%}")

    return pca_features, pca, cum_var

def create_visualizations(pca_features, p_values, cum_var):
    """Создание графиков PCA"""
    # График PCA
    plt.figure(figsize=(8, 6))
    plt.scatter(pca_features[:, 0], pca_features[:, 1], c=p_values, cmap='viridis', alpha=0.6)
    plt.colorbar(label='pKi')
    plt.xlabel('Главная компонента 1')
    plt.ylabel('Главная компонента 2')
    plt.title('Анализ молекулярного сходства (PCA)')
    plt.savefig('pca_plot.png', dpi=150, bbox_inches='tight')
    plt.close()

    # График дисперсии
    plt.figure(figsize=(8, 5))
    plt.plot(cum_var, 'o-')
    plt.axhline(PCA_VARIANCE_THRESHOLD, color='r', linestyle='--', label='Порог объясненной дисперсии')
    plt.axvline(np.argmax(cum_var >= PCA_VARIANCE_THRESHOLD), color='g', linestyle=':', label='Выбранное число компонент')
    plt.xlabel('Количество компонент')
    plt.ylabel('Накопленная дисперсия')
    plt.title('Объясненная дисперсия PCA')
    plt.legend()
    plt.savefig('pca_variance.png', dpi=150, bbox_inches='tight')
    plt.close()

# ───────────────────────── 5. Сохранение результатов ─────────────────────────
def save_results(df, pca_features):
    """Сохранение датасета и визуализаций"""
    print_header("Сохранение результатов")

    # Создание DataFrame из PCA компонент
    pca_columns = [f'pca_{i+1}' for i in range(pca_features.shape[1])]
    pca_df = pd.DataFrame(pca_features, columns=pca_columns)

    # Объединение данных (исправленная версия)
    final_df = pd.concat(
        [
            df[['molecule_chembl_id', 'canonical_smiles', 'pValue']],
            pca_df
        ],
        axis=1
    )

    final_df.to_csv('final_dataset.csv', index=False)

    # Создание графиков
    pca = PCA(n_components=pca_features.shape[1])
    pca.fit(pca_features)
    cum_var = np.cumsum(pca.explained_variance_ratio_)

    create_visualizations(pca_features, df['pValue'], cum_var)

    print(f"Сохранено записей в датасет: {len(final_df)}")
    print("Созданы графики:")
    print("- pca_plot.png (2D визуализация PCA)")
    print("- pca_variance.png (график объясненной дисперсии)")

# ───────────────────────── 6. Основной пайплайн ─────────────────────────
def main():
    """Запуск всего пайплайна обработки данных"""
    print_header("Начало обработки данных QSAR")

    try:
        # 1. Загрузка данных
        raw_df = load_chembl_data()
        if raw_df.empty:
            raise ValueError("Не удалось загрузить данные из ChEMBL")

        # 2. Предобработка
        clean_df = preprocess_data(raw_df)
        if clean_df.empty:
            raise ValueError("Нет данных после предобработки")

        # 3. Генерация отпечатков
        fp_df, fps, _ = generate_fingerprints(clean_df)
        if fp_df.empty:
            raise ValueError("Не удалось сгенерировать отпечатки")

        # 4. PCA
        pca_features, _, _ = apply_pca(fps)
        if pca_features.size == 0:
            raise ValueError("Ошибка при выполнении PCA")

        # 5. Сохранение результатов
        save_results(fp_df, pca_features)

        print_header("Обработка данных завершена успешно!")
        return fp_df, pca_features

    except Exception as e:
        print(f"\nОШИБКА: {str(e)}")
        return None, None

if __name__ == "__main__":
    final_data, _ = main()
    if final_data is not None:
        print("\nПервые 5 записей финального датасета:")
        print(final_data.head())


НАЧАЛО ОБРАБОТКИ ДАННЫХ QSAR

ЗАГРУЗКА ДАННЫХ ИЗ CHEMBL
Получено записей с сервера: 510

ПРЕДОБРАБОТКА ДАННЫХ
Исходное количество записей: 510
Удалено дубликатов SMILES: 42
Удалено записей с нечисловыми значениями: 0
Удалено записей с неположительными значениями: 0
Осталось валидных записей: 465
Диапазон pKi: 4.20 - 10.15

ГЕНЕРАЦИЯ МОЛЕКУЛЯРНЫХ ОТПЕЧАТКОВ
Успешно обработано молекул: 465
Невалидных SMILES: 0
Размерность до фильтрации: 1024 признаков
Размерность после фильтрации: 571 признаков

АНАЛИЗ ГЛАВНЫХ КОМПОНЕНТ (PCA)
Исходное количество признаков: 571
Выбрано главных компонент: 138
Объясненная дисперсия: 95.1%

СОХРАНЕНИЕ РЕЗУЛЬТАТОВ
Сохранено записей в датасет: 465
Созданы графики:
- pca_plot.png (2D визуализация PCA)
- pca_variance.png (график объясненной дисперсии)

ОБРАБОТКА ДАННЫХ ЗАВЕРШЕНА УСПЕШНО!

Первые 5 записей финального датасета:
                                    canonical_smiles molecule_chembl_id  \
0  O=C(c1ccc(OCCN2CCCCC2)cc1)N1c2ccc(O)cc2CCC1c1c...        CH